# 1. Import Required Libraries and Initialize SparkSession
Import `SparkSession` from `pyspark.sql` and create a Spark session.

In [1]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Chapter 2 Example").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/05 16:01:15 WARN Utils: Your hostname, codespaces-1164d4, resolves to a loopback address: 127.0.0.1; using 10.0.1.55 instead (on interface eth0)
25/07/05 16:01:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/05 16:01:15 WARN Utils: Your hostname, codespaces-1164d4, resolves to a loopback address: 127.0.0.1; using 10.0.1.55 instead (on interface eth0)
25/07/05 16:01:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.

# 2. Create a DataFrame with a Range of Numbers
Use `spark.range(1000)` to build a DataFrame of numbers from 0 to 999 and show its schema.

In [3]:
myRange = spark.range(1000).toDF("number")
myRange.printSchema()
myRange.show(5)

root
 |-- number: long (nullable = false)

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
+------+
only showing top 5 rows
+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
+------+
only showing top 5 rows


# 3. Filter DataFrame for Even Numbers
Select only even numbers using a `where` clause.

In [5]:
divisBy2 = myRange.where("number % 2 = 0")
divisBy2.show(5)

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
+------+
only showing top 5 rows


# 4. Read CSV Data into DataFrame
Load the 2015 flight-summary CSV with inferred schema and header.

In [8]:
flightData2015 = (
    spark.read
    .option("inferSchema", "true")
    .option("header", "true")
    .csv("../data/flight-data/csv/2015-summary.csv")
)
flightData2015.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



# 5. Create Temporary SQL View
Register the DataFrame as a temp view for SQL queries.

In [9]:
flightData2015.createOrReplaceTempView("flight_data_2015")

# 6. Group and Count Using SQL and DataFrame APIs
Count the number of flights per destination with both SQL and DataFrame methods.

In [10]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1) AS cnt
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")
dataFrameWay = flightData2015.groupBy("DEST_COUNTRY_NAME").count()
sqlWay.show(5)
dataFrameWay.show(5)

+-----------------+---+
|DEST_COUNTRY_NAME|cnt|
+-----------------+---+
|         Anguilla|  1|
|           Russia|  1|
|         Paraguay|  1|
|          Senegal|  1|
|           Sweden|  1|
+-----------------+---+
only showing top 5 rows
+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
|         Anguilla|    1|
|           Russia|    1|
|         Paraguay|    1|
|          Senegal|    1|
|           Sweden|    1|
+-----------------+-----+
only showing top 5 rows


# 7. Explain Query Execution Plans
Use `explain()` to inspect physical plans.

In [11]:
print("SQL plan:")
sqlWay.explain()
print("DataFrame plan:")
dataFrameWay.explain()

SQL plan:
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#27, 200), ENSURE_REQUIREMENTS, [plan_id=153]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/workspaces/Spark-The-Definitive-Guide/data/flight-data/csv/2015-..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


DataFrame plan:
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#27, 200), ENSURE_REQUIREMENTS, [plan_id=166]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#27] Batched: false, DataFilters: [

# 8. Find Maximum Value in a Column
Use `pyspark.sql.functions.max` to get the max of the `count` column.

In [12]:
from pyspark.sql.functions import max
flightData2015.select(max("count")).show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+



# 9. Aggregate and Sort Data Using SQL
Sum counts by destination, sort descending, limit to top 5.

In [13]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) AS destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY destination_total DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



# 10. Aggregate and Sort Data Using DataFrame API
Perform the same aggregation in the DataFrame API.

In [15]:
from pyspark.sql.functions import desc
flightData2015.groupBy("DEST_COUNTRY_NAME") \
    .sum("count") \
    .withColumnRenamed("sum(count)", "destination_total") \
    .sort(desc("destination_total")) \
    .limit(5) \
    .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



# 11. Explain Aggregation Query Execution Plan
Inspect the physical plan for the DataFrame aggregation+sort query.

In [16]:
(
    flightData2015
    .groupBy("DEST_COUNTRY_NAME")
    .sum("count")
    .withColumnRenamed("sum(count)", "destination_total")
    .sort(desc("destination_total"))
    .limit(5)
    .explain()
)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#108L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#27,destination_total#108L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[sum(count#29)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#27, 200), ENSURE_REQUIREMENTS, [plan_id=307]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[partial_sum(count#29)])
            +- FileScan csv [DEST_COUNTRY_NAME#27,count#29] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/workspaces/Spark-The-Definitive-Guide/data/flight-data/csv/2015-..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




# 12. View Spark Web UI
Get the Spark UI URL for this session and open it in your browser.

In [4]:
# 12a. Retrieve and display Spark Web UI URL
ui_url = spark.sparkContext.uiWebUrl
print("Spark UI available at:", ui_url)
from IPython.display import HTML
HTML(f"<a href='{ui_url}' target='_blank'>{ui_url}</a>")

Spark UI available at: http://e6daf8a3-3328-4fd7-8a84-171ec9947db1.internal.cloudapp.net:4040
